In [93]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
import scipy
import functools as ft
import random

X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
I = np.identity(2)

In [94]:
n = 10
b = [random.random()*3 for _ in range(n)]

def get_gi(gate, i, get_gi1=False):
    gates = []
    for j in range(n):
        if j == i or (j == i+1 and get_gi1):
            gates.append(gate)
        else:
            gates.append(I)
    return ft.reduce(np.kron, gates)

h1 = sum((get_gi(X, i, True) + get_gi(Y, i, True)) for i in range(n-1))

h2 = sum((b[i] * get_gi(Z, i)) for i in range(n))

h = h1 + h2

def u(t):
    return scipy.linalg.expm(-1j * h * t)

#print(h1)
#print(h)

In [95]:
simulator = Aer.get_backend('aer_simulator')

for i in np.arange(0, 10, .5):
    qc = QuantumCircuit(n)
    qc.x(0) # create |100...0> state

    qc.unitary(u(i), range(n))
    qc.measure_all()

    compiled_circuit = transpile(qc, simulator)
    job = simulator.run(compiled_circuit, shots=1000)
    result = job.result()
    counts = result.get_counts()

    print(counts.most_frequent())

0000000001
0000000010
0000000100
0000001000
0000000100
0000000001
0000000001
0000000010
0000001000
0000000100
0000000001
0000000001
0000000010
0000001000
0000000010
0000000001
0000000001
0000000100
0000001000
0000000100
